In [ ]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from scipy.stats import uniform
from sklearn.metrics import  make_scorer
from sklearn.utils.fixes import loguniform
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import validation_curve

In [ ]:
#cargo los datos en el dataframe
dataframe=pd.read_csv('/content/100-fosfo-343.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'FOSFOI'], axis='columns')
print(dataframe)

    OXA-64  rosA  rosA_cp1  macB  ...  OXA-104  OXA-132  OXA-180  FOSFOMICINA_
0      1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0          2048
1      1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0          4096
2     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0          4096
3     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0          2048
4     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0          4096
..     ...   ...       ...   ...  ...      ...      ...      ...           ...
81    -1.0   7.0      -1.0   4.0  ...     -1.0     -1.0     -1.0            64
82    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0            64
83    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0            64
84    -1.0   9.0      -1.0   1.0  ...     -1.0     -1.0     -1.0            64
85    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0            64

[86 rows x 212 columns]


In [ ]:
#defino las variables "X" y "y"
X = dataframe.drop(['FOSFOMICINA_'], axis='columns')
y = dataframe.iloc[:, -1] 
print(X)
print(y)

    OXA-64  rosA  rosA_cp1  macB  ...  OXA-431  OXA-104  OXA-132  OXA-180
0      1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
1      1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
2     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
3     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
4     -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
..     ...   ...       ...   ...  ...      ...      ...      ...      ...
81    -1.0   7.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
82    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0     -1.0
83    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0     -1.0
84    -1.0   9.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
85    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0     -1.0

[86 rows x 211 columns]
0     2048
1     4096
2     4096
3     2048
4     4096
      ... 
81      64
82      64

In [ ]:
#cambio de variables a categoricas por dummys 
datadummy = pd.get_dummies(X.astype(str))
print(datadummy)

    OXA-64_-1.0  OXA-64_1.0  rosA_2.0  ...  OXA-104_-1.0  OXA-132_-1.0  OXA-180_-1.0
0             0           1         0  ...             1             1             1
1             0           1         0  ...             1             1             1
2             1           0         0  ...             1             1             1
3             1           0         0  ...             1             1             1
4             1           0         0  ...             1             1             1
..          ...         ...       ...  ...           ...           ...           ...
81            1           0         0  ...             1             1             1
82            1           0         1  ...             1             1             1
83            1           0         1  ...             1             1             1
84            1           0         0  ...             1             1             1
85            1           0         1  ...             1         

In [ ]:
#Eliminar columnas que indican la ausencia del gen 
for col in datadummy.columns: 
  if col[-4:] == '-1.0':
    print(col)
    del(datadummy[col])

print(datadummy.columns)
print(datadummy.shape)

OXA-64_-1.0
rosA_cp1_-1.0
Acinetobacter_baumannii_AmvA_-1.0
Acinetobacter_baumannii_AmvA_cp1_-1.0
adeN_-1.0
adeH_-1.0
adeF_cp1_-1.0
adeL_-1.0
abeS_-1.0
abeS_cp1_-1.0
ANT(3'')-IIc_-1.0
ANT(3'')-IIc_cp1_-1.0
adeB_-1.0
adeB_cp1_-1.0
adeA_-1.0
adeA_cp1_-1.0
adeR_-1.0
adeR_cp1_-1.0
adeS_-1.0
adeS_cp1_-1.0
ADC-5_-1.0
MexK_-1.0
Mycobacterium_tuberculosis_rpoB_mutants_conferring_resistance_to_rifampicin_-1.0
adeJ_-1.0
Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone_-1.0
Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone_cp1_-1.0
Escherichia_coli_soxR_with_mutation_conferring_antibiotic_resistance_-1.0
sul2_-1.0
sul2_cp1_-1.0
sul2_cp2_-1.0
tet(B)_-1.0
APH(6)-Id_-1.0
APH(6)-Id_cp1_-1.0
APH(3'')-Ib_-1.0
APH(3'')-Ib_cp1_-1.0
OXA-23_-1.0
OXA-23_cp1_-1.0
AAC(3)-IIe_-1.0
AAC(6')-Ian_-1.0
TEM-1_-1.0
Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones_-1.0
Mycobacterium_tuberculosis_thyA_with_mutation_conferring_resistance_to_para-aminosalicylic_ac

In [ ]:
#division de los datos en entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(datadummy, y, test_size=0.20, random_state=27)
print(X_train)
print(y_test)

In [ ]:
#convierto las variables de "y" en log2
y_train1 = np.log2(y_train)
y_test1 = np.log2(y_test)
print(y_test1)
print(y_train1)

59    11.0
30     7.0
21    10.0
20     8.0
60    10.0
67     7.0
22    11.0
71     9.0
55    11.0
47     9.0
6      7.0
53    12.0
83     6.0
69    12.0
14    11.0
77     6.0
79     6.0
52    11.0
Name: FOSFOMICINA_, dtype: float64
1     12.0
11    11.0
38    12.0
45     9.0
50    12.0
      ... 
84     6.0
31     7.0
72     7.0
56    12.0
19     7.0
Name: FOSFOMICINA_, Length: 68, dtype: float64


In [ ]:
# find optimal alpha with grid search: REGULARIZACION
lasso = Lasso()
alpha = [0.001, 0.005, 0.01, 0.05, 0.06, 0.07, 0.08, 0.085, 0.086, 0.087, 0.088, 0.089, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(estimator=lasso, param_grid=param_grid, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1, cv=10)
grid_result = grid.fit(X_train, y_train1)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

In [ ]:
#defino el modelo y los valores de alpha a evaluar (distantes)
parametros = [0.001, 0.005, 0.009, 0.01, 0.03, 0.05, 0.1]

In [ ]:
#defino el modelo y los valores de alpha a evaluar (cortos)
parametros = [0.005, 0.01, 0.03, 0.05, 0.07, 0.1, 0.3, 0.7]


In [ ]:
#tunning de hiperparametros (resultados con r2)
train_scores, valid_scores = validation_curve(Lasso(), X = X_train, y=y_train1, param_name ="alpha", param_range= parametros, cv=10, scoring="r2")
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(valid_scores, axis=1)
val_std = np.std(valid_scores, axis=1)
print('Score train: ', train_scores)
print('Score test: ', valid_scores)
print('RMSE train: ', train_mean)
print('RMSE val: ', val_mean)

In [ ]:
#grafica de validacion R2
plt.plot(parametros, train_mean, color='blue', marker='o', markersize=5, label='training score')
plt.fill_between(parametros, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(parametros, val_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation score')
plt.fill_between(parametros, val_mean + val_std, val_mean - val_std, alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
#plt.xscale('linear')
#plt.xticks(parametros)
plt.legend(loc='upper righ')
plt.xlabel('Alpha')
plt.ylabel('Score')
plt.ylim([-1, 1.1])
plt.show()

In [ ]:
#tunning de hiperparametros (resultados con RMSE)
train_scores, valid_scores = validation_curve(Lasso(), X = X_train, y=y_train1, param_name ="alpha", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
train_scores_positive = np.array(train_scores) * -1
valid_scores_positive = np.array(valid_scores) * -1
train_mean = np.mean(train_scores_positive, axis=1)
train_std = np.std(train_scores_positive, axis=1)
val_mean = np.mean(valid_scores_positive, axis=1)
val_std = np.std(valid_scores_positive, axis=1)
print('Score train: ', train_scores_positive)
print('Score test: ', valid_scores_positive)
print('RMSE train: ', train_mean)
print('RMSE val: ', val_mean)

In [ ]:
#grafica de validacion RMSE
plt.plot(parametros, train_mean, color='blue', marker='o', markersize=5, label='training score')
plt.fill_between(parametros, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(parametros, val_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation score')
plt.fill_between(parametros, val_mean + val_std, val_mean - val_std, alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
#plt.xscale('linear')
#plt.xticks(parametros)
plt.legend(loc='upper righ')
plt.xlabel('Alpha')
plt.ylabel('Score')
plt.ylim([0, 3])
plt.show()

In [ ]:
#entrenamos el modelo con el valor de alpha establecido por gride search 
lasso001 = Lasso(alpha=0.05, max_iter=10e5)
lasso001.fit(X_train, y_train1)
train_score = lasso001.score(X_train, y_train1)
test_score = lasso001.score(X_test, y_test1)
coeff_used = np.sum(lasso001.coef_!=0)
print("train score:", train_score) #r2 del entrenamiento 
print("test score:", test_score) #r2 del test
print("numero de caracteristicas usadas:", coeff_used)


In [ ]:
#hacer prediccion 
print("Predicciones")
predictions = 2**np.round(lasso001.predict(X_test))
print(predictions)
verdaderos = y_test.to_numpy()
print(verdaderos)
# Por último, calculamos el error cuadrático medio y el estadístico R^2
print ('Error cuadrático medio: %.2f' % mean_squared_error(y_test1, np.log2(predictions)))
print ('Estadístico R_2: %.2f' % r2_score(y_test1, np.log2(predictions)))
print(mean_absolute_error(y_test1, np.log2(predictions)))

Predicciones
[ 512.  512. 1024. 1024. 1024.  512. 1024. 1024. 1024.  256.  512. 1024.
  128.  512. 1024.  128.   64. 1024.]
[2048  128 1024  256 1024  128 2048  512 2048  512  128 4096   64 4096
 2048   64   64 2048]
Error cuadrático medio: 2.28
Estadístico R_2: 0.49
1.2777777777777777


In [ ]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y_test, predictions, dill=2):
    frac = np.array(y_test) / np.array(predictions)
    return ((frac >= 1/dill) & (frac <= dill)).mean()
   

In [ ]:
acc2 = within_dillution(y_test, predictions, dill=2)
acc4 = within_dillution(y_test, predictions, dill=4)
print(acc2)
print(acc4)

0.6111111111111112
0.9444444444444444


In [ ]:
threshold = 2 # estos valores correspondientes a los puntos de corte del para clasificar como suceptible un genoma (varia segun antibiotico)
threshold1 = 2
predDF = pd.DataFrame({'predictions':predictions, 'y_test':y_test})
vme = len(predDF[(predDF.predictions<=threshold) & (predDF.y_test>threshold1)])
me = len(predDF[(predDF.predictions>threshold1) & (predDF.y_test<=threshold)])
print(vme)
print(me)

0
0
